# A simple Tensorflow example

### This code is intended to serve you as a starting point to build Tensorflow code for your own purposes. The examples we will cover will be applied to the case of classification, but some of the principles and tools will be useful for regression also, as well as for unsupervised learning methods such as autoencoders or GANs.


#### Requirements:
 - numpy
 - matplotlib
 - tensorflow –if you need info on how to install Tensorflow, please follow the steps on the documentation: https://www.tensorflow.org/install/

##### First import some useful packages

In [ ]:

import numpy as np
import tensorflow as tf
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

###### Tensorflow likes to use one-hot encoding for categorical variables (a.k.a. dummies in scikit-learn, for instance).

In [ ]:
def int2onehot(t_in, num_class, valid_targets):
    '''
    Convert target value from integer to one-hot.
    
     PARAMETERS
        t_in           : (N)x(1) np.array of target values (integers)
        num_class      : number of classes & size of one-hot vector 
        valid_targets  : list or np.array of valid target values
    
      RETURNS
        t_out          : (N)x(num_class) np.array of one-hot vectors 
    '''

    
    # Construct available target values if not passed
    if valid_targets == ():
        valid_targets = list(set(t_in))

    
    # Number of data points 
    N=np.size(t_in)
    
    
    # Initialize to zero and change value where needed
    t_out=np.zeros((N, num_class), dtype=int)
    for i in range(0,N):
        # For this data point, find class
        TrueOrFalse = (valid_targets == t_in[i])
                   
        # Convert list of True/False into 1/0 and
        # save into output array of one-hot vectors.
        t_out[i] = 1*TrueOrFalse

        
    return t_out

###### Read data from CSV file into a numpy array

In [ ]:
datadir='./'
datafile='donut_2000.csv'
data=np.loadtxt(datadir + datafile, delimiter=',')

###### Rearrange data: 
- separate features and target values
- split into training and test sets
- one-hot encoding of targets

In [ ]:
# Separate features and target
ifeat=[0,1,2]                    # Indices of feature values
itgt = 3                         # Index of target value
n_classes = 2                    # Number of classes
Nfeat=len(ifeat)

x_all=data[:,ifeat]     # features
t_all=data[:,itgt]      # target





# Split into training and test sets, shuffling data points
testpercent = 20
Ndata=np.size(t_all)
Ntes=Ndata*testpercent/100
Ntra=Ndata-Ntes

ishuff=np.arange(0,Ndata)
np.random.shuffle(ishuff)
itra=ishuff.tolist()[0:Ntra]
ites=ishuff.tolist()[Ntra:Ntra+Ntes]

x_train = x_all[itra,:]
t_train = t_all[itra]
x_test  = x_all[ites,:]
t_test  = t_all[ites]



# one-hot enconding of target
valid_target_values=[0,1]
t_train = int2onehot(t_train.reshape(Ntra), n_classes, valid_target_values)
t_test  = int2onehot(t_test.reshape(Ntes), n_classes, valid_target_values)

##### Let's visualize the training data!

In [ ]:
plotdata = True
if plotdata:
    from mpl_toolkits.mplot3d import Axes3D

    fs = 15   # fontsize
        
    # Select indices of 0 and 1 classes
    i0 = t_train[:,0]==1
    i1 = t_train[:,1]==1
    
    # 3d scatter plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')


    

    xs = x_train[:,0]
    ys = x_train[:,1]
    zs = x_train[:,2]
    
    
    ax.scatter(xs[i0], ys[i0], zs[i0], c='r', s=1, alpha=0.2)#, marker=m)
    ax.scatter(xs[i1], ys[i1], zs[i1], c='b', s=1, alpha=0.8)#, marker=m)

    ax.set_xlabel('x', fontsize=fs)
    ax.set_ylabel('y', fontsize=fs)
    ax.set_zlabel('z', fontsize=fs)
    plt.show();
    
    
    
    # xy plot
    plt.clf()
    plt.figure(figsize=(5,5))
    plt.scatter(xs[i0], ys[i0], c='r', s=1, alpha=0.2)
    plt.scatter(xs[i1], ys[i1], c='b', s=1, alpha=0.8)
    plt.xlabel('x', fontsize=fs)
    plt.ylabel('y', fontsize=fs)
    plt.show();
    
    
    
    # xz plot
    plt.clf()
    plt.figure(figsize=(5,2))
    plt.scatter(xs[i0], zs[i0], c='r', s=1, alpha=0.2)
    plt.scatter(xs[i1], zs[i1], c='b', s=1, alpha=0.8)
    plt.xlabel('x', fontsize=fs)
    plt.ylabel('z', fontsize=fs)
    plt.show();

##### It's a donut. Yum! --the notebook data_generator.ipynb contains the code that generated this data set

###### Let us start with a super simple example: a network with only one neuron

In [ ]:
##########################
# Single neuron classifier

# Learning parameters
learning_rate = 0.001
training_epochs = 200
batch_size = 250

# Optimization

opt_type = 'Adam' # 'GD' # 


# Printout options
display_step = 25


train_num_examples = Ntra


# Network Parameters
n_input = Nfeat       # Number of features
n_classes = 2         # total classes 


# Create tensor placeholders for features and targets
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# NEURAL NET DEFINITION
#
# Layers weights and biases are initialized with normally
# distributed random values.
weights = {
    'h': tf.Variable(tf.random_normal([n_input,1])),
    'out': tf.Variable(tf.random_normal([1, n_classes]))
}
biases = {
    'b': tf.Variable(tf.random_normal([1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
#
# Network topology
def perceptron(x, weights, biases):
    # Single hidden neuron with sigmoid activation
    neuron = tf.add(tf.matmul(x, weights['h']), biases['b'])
    neuron = tf.nn.sigmoid(neuron)
    # Output layer with linear activation
    out_layer = tf.matmul(neuron, weights['out']) + biases['out']
    return out_layer



# Python list of model internal parameters
params  = tf.trainable_variables()


# Construct model
pred = perceptron(x, weights, biases)


# Define cost function: cross-entropy of the softmax 
# probability distribution on the output layer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))


# Cost function optimization routine
if opt_type == 'GD':
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
elif opt_type == 'Adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
else:
    print("ERROR: optimizer type unknown. Exiting...")
    exit()


# Method that evaluates if a prediction is correct.
# Take output unit 
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))



# METRICS
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))



# Initialize variables
init = tf.global_variables_initializer()


epochs = []
costs = []
train_accs = []
test_accs = []
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(train_num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            iL_batch = i*batch_size
            iR_batch = (i+1)*batch_size
            batch_x = x_train[iL_batch:iR_batch, :]
            batch_y = t_train[iL_batch:iR_batch, :]

            # Run optimization (backpropagation) and cost (to get loss value)
            _, c, = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            # Compute average loss in batch
            avg_cost += c / total_batch
            
            
        # Display logs per epoch step
        if epoch % display_step == 0:

            train_acc = accuracy.eval({x: x_train, y: t_train}) 
            test_acc = accuracy.eval({x: x_test,   y: t_test})
            print("Epoch:", epoch, "  ", 
                  "cost=", round(avg_cost,4), "  ", 
                  "train acc.=", train_acc, "  ",  
                  "test acc.=", test_acc)
            
            # Save metrics to evaluate convergence
            epochs.append(epoch)
            costs.append(avg_cost) 
            train_accs.append(train_acc)
            test_accs.append(test_acc)
            
            
    
    print("Optimization Finished!")
    
    
    # Calculate accuracy and bias on test set
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Train accuracy:", accuracy.eval({x: x_train, y: t_train}))
    print("Test accuracy :", accuracy.eval({x: x_test,  y: t_test}))
    

    
# Convergence plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6), sharey=False)
ax1.plot(epochs, costs)
ax1.set_xlabel('epoch')
ax1.set_ylabel('cost')

ax2.plot(epochs, train_accs, label='train')
ax2.plot(epochs, test_accs, label='test')
ax2.set_xlabel('epoch')
ax2.set_ylabel('accuracy')
ax2.legend()

In [ ]:
#####################################################
# Neural net with 1 hidden layer

# Learning parameters
learning_rate = 0.001
training_epochs = 3000
batch_size = 50

# Optimization

opt_type = 'Adam' # 'GD' # 


# Printout options
display_step = 100


train_num_examples = Ntra


# Network Parameters
n_input = Nfeat       # Number of features
n_hidden_1 = 500      # 1st layer number of features
n_classes = 2         # total classes 



# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])



# NEURAL NET DEFINITION
# Layers weights and biases
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Network
def single_layer_net(x, weights, biases):
    # Hidden layer with sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer



# List of model internal parameters
params  = tf.trainable_variables()


# Construct model
pred = single_layer_net(x, weights, biases)


# Define cost function:
# cross-entropy of the softmax probability distribution on the output layer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))


# Cost function optimization routine
if opt_type == 'GD':
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
elif opt_type == 'Adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
else:
    print("ERROR: optimizer type unknown. Exiting...")
    exit()


    
# Method that evaluates if a prediction is correct
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))



# METRICS
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))



# Initialize variables
init = tf.global_variables_initializer()


epochs = []
costs = []
train_accs = []
test_accs = []
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(train_num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            iL_batch = i*batch_size
            iR_batch = (i+1)*batch_size
            batch_x = x_train[iL_batch:iR_batch, :]
            batch_y = t_train[iL_batch:iR_batch, :]

            # Run optimization (backpropagation) and cost (to get loss value)
            _, c, = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            # Compute average loss in batch
            avg_cost += c / total_batch
            
            
        # Display logs per epoch step
        if epoch % display_step == 0:

            train_acc = accuracy.eval({x: x_train, y: t_train}) 
            test_acc = accuracy.eval({x: x_test,   y: t_test})
            print("Epoch:", epoch, "  ", 
                  "cost=", round(avg_cost,4), "  ", 
                  "train acc.=", train_acc, "  ",  
                  "test acc.=", test_acc)
            
            # Save metrics to evaluate convergence
            epochs.append(epoch)
            costs.append(avg_cost) 
            train_accs.append(train_acc)
            test_accs.append(test_acc)
            
    
    print("Optimization Finished!")
    
    
    # Calculate accuracy and bias on test set
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Train accuracy:", accuracy.eval({x: x_train, y: t_train}))
    print("Test accuracy :", accuracy.eval({x: x_test,  y: t_test}))
    
    
# Convergence plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6), sharey=False)
ax1.plot(epochs, costs)
ax1.set_xlabel('epoch')
ax1.set_ylabel('cost')

ax2.plot(epochs, train_accs, label='train')
ax2.plot(epochs, test_accs, label='test')
ax2.set_xlabel('epoch')
ax2.set_ylabel('accuracy')
ax2.legend()

In [ ]:
##################
# 2-layer network

# Learning parameters
learning_rate = 0.001
training_epochs = 250
batch_size = 100

# Optimization

opt_type = 'Adam' # 'GD' # 


# Printout options
display_step = 25


train_num_examples = Ntra


# Network Parameters
n_input = Nfeat       # Number of features
n_hidden_1 = 5        # 1st layer number of units
n_hidden_2 = 5        # 2st layer number of units
n_classes  = 2        # total classes 



# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])



# NEURAL NET DEFINITION
# Layers weights and biases
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_input, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Network
def two_layer_net(x, weights, biases):
    # Hidden layer with sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with ReLu activation
    layer_2 = tf.add(tf.matmul(x, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer



# List of model internal parameters
params  = tf.trainable_variables()


# Construct model
pred = two_layer_net(x, weights, biases)


# Define cost function:
# cross-entropy of the softmax probability distribution on the output layer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))


# Cost function optimization routine
if opt_type == 'GD':
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
elif opt_type == 'Adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
else:
    print("ERROR: optimizer type unknown. Exiting...")
    exit()


    
# Method that evaluates if a prediction is correct
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))



# METRICS
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))



# Initialize variables
init = tf.global_variables_initializer()


epochs = []
costs = []
train_accs = []
test_accs = []
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(train_num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            iL_batch = i*batch_size
            iR_batch = (i+1)*batch_size
            batch_x = x_train[iL_batch:iR_batch, :]
            batch_y = t_train[iL_batch:iR_batch, :]

            # Run optimization (backpropagation) and cost (to get loss value)
            _, c, = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            # Compute average loss in batch
            avg_cost += c / total_batch
            
            
        # Display logs per epoch step
        if epoch % display_step == 0:

            train_acc = accuracy.eval({x: x_train, y: t_train}) 
            test_acc = accuracy.eval({x: x_test,   y: t_test})
            print("Epoch:", epoch, "  ", 
                  "cost=", round(avg_cost,4), "  ", 
                  "train acc.=", train_acc, "  ",  
                  "test acc.=", test_acc)
            
            # Save metrics to evaluate convergence
            epochs.append(epoch)
            costs.append(avg_cost) 
            train_accs.append(train_acc)
            test_accs.append(test_acc)
            
    
    print("Optimization Finished!")
    
    
    # Calculate accuracy and bias on test set
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Train accuracy:", accuracy.eval({x: x_train, y: t_train}))
    print("Test accuracy :", accuracy.eval({x: x_test,  y: t_test}))
    
    
# Convergence plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6), sharey=False)
ax1.plot(epochs, costs)
ax1.set_xlabel('epoch')
ax1.set_ylabel('cost')

ax2.plot(epochs, train_accs, label='train')
ax2.plot(epochs, test_accs, label='test')
ax2.set_xlabel('epoch')
ax2.set_ylabel('accuracy')
ax2.legend()

In [ ]:
##################
# 3-layer network

# Learning parameters
learning_rate = 0.001
training_epochs = 250
batch_size = 100

# Optimization

opt_type = 'Adam' # 'GD' # 


# Printout options
display_step = 25


train_num_examples = Ntra


# Network Parameters
n_input = Nfeat       # Number of features
n_hidden_1 = 4        # 1st layer number of units
n_hidden_2 = 7        # 2st layer number of units
n_hidden_3 = 3        # 3rd layer number of units
n_classes = 2         # total classes 



# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])



# NEURAL NET DEFINITION

# Layers weights and biases
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Net
def three_layer_net(x, weights, biases):
    # Hidden layer with sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with ReLU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Hidden layer with sigmoid activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer






# List of model internal parameters
params  = tf.trainable_variables()


# Construct model
pred = three_layer_net(x, weights, biases)


# Define cost function:
# cross-entropy of the softmax probability distribution on the output layer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))


# Cost function optimization routine
if opt_type == 'GD':
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
elif opt_type == 'Adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
else:
    print("ERROR: optimizer type unknown. Exiting...")
    exit()


    
# Method that evaluates if a prediction is correct
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))



# METRICS
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))



# Initialize variables
init = tf.global_variables_initializer()

epochs = []
costs = []
train_accs = []
test_accs = []
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(train_num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            iL_batch = i*batch_size
            iR_batch = (i+1)*batch_size
            batch_x = x_train[iL_batch:iR_batch, :]
            batch_y = t_train[iL_batch:iR_batch, :]

            # Run optimization (backpropagation) and cost (to get loss value)
            _, c, = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            # Compute average loss in batch
            avg_cost += c / total_batch
            
            
        # Display logs per epoch step
        if epoch % display_step == 0:

            train_acc = accuracy.eval({x: x_train, y: t_train}) 
            test_acc = accuracy.eval({x: x_test,   y: t_test})
            print("Epoch:", epoch, "  ", 
                  "cost=", round(avg_cost,4), "  ", 
                  "train acc.=", train_acc, "  ",  
                  "test acc.=", test_acc)
            
            # Save metrics to evaluate convergence
            epochs.append(epoch)
            costs.append(avg_cost) 
            train_accs.append(train_acc)
            test_accs.append(test_acc)
            
    
    print("Optimization Finished!")
    
    
    # Calculate accuracy and bias on test set
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Train accuracy:", accuracy.eval({x: x_train, y: t_train}))
    print("Test accuracy :", accuracy.eval({x: x_test,  y: t_test}))
    
    

# Convergence plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6), sharey=False)
ax1.plot(epochs, costs)
ax1.set_xlabel('epoch')
ax1.set_ylabel('cost')

ax2.plot(epochs, train_accs, label='train')
ax2.plot(epochs, test_accs, label='test')
ax2.set_xlabel('epoch')
ax2.set_ylabel('accuracy')
ax2.legend()

###### Remember that the 1-layer network was overfitting? L2 regularization is one way to tackle this issue. Let's see how to implement it. Pay close attention to the part of the code were the cost function is built.

In [ ]:
##################
# 1-layer network with L2 regularization

# Learning parameters
learning_rate = 0.001
training_epochs = 2000
batch_size = 50

# Optimization
opt_type = 'Adam' # 'GD' # 

# Regularization parameter
beta = 0.01

# Printout options
display_step = 100



train_num_examples = Ntra


# Network Parameters
n_input = Nfeat       # Number of features
n_hidden_1 = 500      # 1st layer number of units
n_classes = 2         # total classes 



# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])



# NEURAL NET DEFINITION

# Layers weights and biases
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Net
def one_layer_net(x, weights, biases):
    # Hidden layer with sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer



# List of model internal parameters
params  = tf.trainable_variables()


# Construct model
pred = one_layer_net(x, weights, biases)


# Define cost function
#
# cross-entropy of the softmax probability distribution on the output layer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

# The L2 regularizer is the sum of the squares of all the weights.
# Since we have several weight vectors, one for each layer, we
# need to sum the L2 terms of each of them
regularizer = sum([tf.nn.l2_loss(weights[key]) for key in weights.keys()])

# Now, add the regularizer to the cost function
cost = cost + beta*regularizer


# Cost function optimization routine
if opt_type == 'GD':
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
elif opt_type == 'Adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
else:
    print("ERROR: optimizer type unknown. Exiting...")
    exit()


    
# Method that evaluates if a prediction is correct
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))



# METRICS
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))



# Initialize variables
init = tf.global_variables_initializer()


epochs = []
costs = []
train_accs = []
test_accs = []
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(train_num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            iL_batch = i*batch_size
            iR_batch = (i+1)*batch_size
            batch_x = x_train[iL_batch:iR_batch, :]
            batch_y = t_train[iL_batch:iR_batch, :]

            # Run optimization (backpropagation) and cost (to get loss value)
            _, c, = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            # Compute average loss in batch
            avg_cost += c / total_batch
            
            
        # Display logs per epoch step
        if epoch % display_step == 0:

            train_acc = accuracy.eval({x: x_train, y: t_train}) 
            test_acc = accuracy.eval({x: x_test,   y: t_test})
            l2reg = regularizer.eval()
            print("Epoch:", epoch, "  ", 
                  "cost=", round(avg_cost,4), "  ", 
                  "L2=", l2reg, "  ", 
                  "train acc.=", train_acc, "  ",  
                  "test acc.=", test_acc)
            
            # Save metrics to evaluate convergence
            epochs.append(epoch)
            costs.append(avg_cost) 
            train_accs.append(train_acc)
            test_accs.append(test_acc)
            
    
    print("Optimization Finished!")
    
    
    # Calculate accuracy and bias on test set
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Train accuracy:", accuracy.eval({x: x_train, y: t_train}))
    print("Test accuracy :", accuracy.eval({x: x_test,  y: t_test}))
    
    
# Convergence plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6), sharey=False)
ax1.plot(epochs, costs)
ax1.set_xlabel('epoch')
ax1.set_ylabel('cost')

ax2.plot(epochs, train_accs, label='train')
ax2.plot(epochs, test_accs, label='test')
ax2.set_xlabel('epoch')
ax2.set_ylabel('accuracy')
ax2.legend()

In [ ]:
# We can also wrap our model evaluation into a function.
# Notice that the data arrays and other variables such
# as the number of features are already in the namespace,
# so we do not need to take them as arguments in the wrapper
# function –although it would be better coding practice.

def run_model(learning_rate=0.001,
              training_epochs=100,
              batch_size=50,
              opt_type='GD',
              beta=0.01,
              display_step=10,
              print_steps=False):
    '''
    Wrapper function for model evaluation
    '''
    
    # Initialize variables
    init = tf.global_variables_initializer()


    epochs = []
    costs = []
    train_accs = []
    test_accs = []
    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)

        # Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.0
            total_batch = int(train_num_examples/batch_size)
        
            # Loop over all batches
            for i in range(total_batch):
                iL_batch = i*batch_size
                iR_batch = (i+1)*batch_size
                batch_x = x_train[iL_batch:iR_batch, :]
                batch_y = t_train[iL_batch:iR_batch, :]

                # Run optimization (backpropagation) and cost (to get loss value)
                _, c, = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
                # Compute average loss in batch
                avg_cost += c / total_batch
            
            
            # Display logs per epoch step
            if epoch % display_step == 0:

                train_acc = accuracy.eval({x: x_train, y: t_train}) 
                test_acc = accuracy.eval({x: x_test,   y: t_test})
                l2reg = regularizer.eval()
                if print_steps:
                    print("Epoch:", epoch, "  ", 
                          "cost=", round(avg_cost,4), "  ", 
                          "L2=", l2reg, "  ", 
                          "train acc.=", train_acc, "  ",  
                          "test acc.=", test_acc)
            
                # Save metrics to evaluate convergence
                epochs.append(epoch)
                costs.append(avg_cost) 
                train_accs.append(train_acc)
                test_accs.append(test_acc)
            
    
        print("Optimization Finished!")
    
    
        # Calculate accuracy and bias on test set
        print("Train accuracy:", accuracy.eval({x: x_train, y: t_train}))
        print("Test accuracy :", accuracy.eval({x: x_test,  y: t_test}))
    
    
    
    # Convergence plots
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6), sharey=False)
    ax1.plot(epochs, costs)
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('cost')

    ax2.plot(epochs, train_accs, label='train')
    ax2.plot(epochs, test_accs, label='test')
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('accuracy')
    ax2.legend()
    
    # Return final train and test accuracies
    return train_accs[-1], test_accs[-1]

In [ ]:
# Now with this wrapper function we can easily script 
# our model evaluation and evaluate the performance
# as parameters change

learning_rate = 0.001
training_epochs = 500
batch_size = 10
opt_type = 'Adam' # 'GD' # 
betas = [0.0, 0.001, 0.01, 0.1]
display_step = 50

train_accs=[]
test_accs =[]
for beta in betas:
    print('beta = ', beta)
    tr_a, te_a = run_model(learning_rate, 
                           training_epochs, 
                           batch_size, 
                           opt_type, 
                           beta, 
                           display_step)
    train_accs.append(tr_a)
    test_accs.append(te_a)

In [ ]:
# Plot train and test accuracies for all betas
plt.plot(range(len(train_accs)), train_accs, marker='x', label='train')
plt.plot(range(len(test_accs)), test_accs, marker='x', label='test')
plt.xlabel('betas', fontsize=15)
plt.ylabel('accuracies', fontsize=15)
plt.xticks(range(len(train_accs)), betas, rotation='vertical')
plt.legend()

## Closing notes
#### We have seen a few examples of neural networks for classification. They were not intended to be a thorough machine learning exercise, but rather a gentle intro to how to use Tensorflow. Hopefully you can use some of the pieces of code that you've seen here to build your own neural nets to perform a rigorous ML analysis. 

#### Have fun! :)